In [ ]:
%pip install yfinance==0.1.74
%pip install mplcyberpunk
%pip install pywin32

In [ ]:
import pandas as pd
import datetime
import yfinance as yf
from matplotlib import pyplot as plt
import mplcyberpunk
import win32com.client as win32

In [ ]:
tickers = ["BTC-USD", "ETH-USD", "^GSPC", "BRL=X"]

hoje = datetime.datetime.now()
um_ano_atras = hoje - datetime.timedelta(days=365)

dados_mercado = yf.download(tickers, um_ano_atras, hoje)

display(dados_mercado)

In [ ]:
dados_fechamento = dados_mercado['Adj Close']


dados_fechamento.columns = ['Dólar', 'Bitcoin', 'Ethereum', 'SP500']

# .dropna() serve p tirar os dados vazios "NaN"
# dados_fechamento = dados_fechamento.dropna()
#primeiros 50 dados
# dados_fechamento.head(50)

dados_fechamento

In [ ]:
#Criando timeframes
# Trocando o Y no resample para D, W, M, Q, Y
# .last() é para pegar o fechamento, poderia ser .sum(), .mean(), e por aí vaí
dados_anuais = dados_fechamento.resample("Y").last()

dados_mensais = dados_fechamento.resample("M").last()

display(dados_anuais, dados_mensais)
# dados_mensais

In [ ]:
# Calculando retorno anual, mensal, diario, semanal
retorno_anual = dados_anuais.pct_change().dropna()

retorno_mensal = dados_mensais.pct_change().dropna()

retorno_diario = dados_fechamento.pct_change().dropna()

retorno_semanal = dados_fechamento.resample("W").last().pct_change().dropna()

display(retorno_anual, retorno_mensal, retorno_semanal , retorno_diario)

In [ ]:
# loc -> referencia elementos a partir do nome
# iloc -> seleciona elementos como uma matriz

# # Exemplos
# retornoDia_2023_02_02 = retorno_diario.loc['2023-02-02', 'Bitcoin']
# # 

# # Matriz[Linha, Coluna] -> Contagem começa a partir do 0
# retornoDia_2023_02_02_iloc = retorno_diario.iloc[1, 3]
# display(retornoDia_2023_02_02, retornoDia_2023_02_02_iloc)



# Loop feito para melhorar a leitura e resumir, transformei em porcentagem ao multiplicar por 100, e arredondei para 2 casas decimais com round(x, 2)
intervals = ['diario', 'semanal', 'mensal', 'anual']
interval_dicts = {}
for interval in intervals:
    retorno_interval = eval("retorno_" + interval)
    retorno_interval_dolar = round(retorno_interval.iloc[-1, 0] * 100, 2)
    retorno_interval_bitcoin = round(retorno_interval.iloc[-1, 1] * 100, 2)
    retorno_interval_ethereum = round(retorno_interval.iloc[-1, 2] * 100, 2)
    retorno_interval_sp500 = round(retorno_interval.iloc[-1, 3] * 100, 2)
    interval_dicts[interval] = {"dolar": retorno_interval_dolar, "bitcoin": retorno_interval_bitcoin,
                                "ethereum": retorno_interval_ethereum, "sp500": retorno_interval_sp500}

# acessando informações de um determinado intervalo (diario) [Exemplo]
interval = "anual"
retorno_interval_dolar = interval_dicts[interval]["dolar"]
retorno_interval_bitcoin = interval_dicts[interval]["bitcoin"]
retorno_interval_ethereum = interval_dicts[interval]["ethereum"]
retorno_interval_sp500 = interval_dicts[interval]["sp500"]


display([retorno_interval_dolar, retorno_interval_bitcoin, retorno_interval_ethereum, retorno_interval_sp500])


In [ ]:
# Criar os gráficos a partir dos dados

# Definindo o tema do grafico
plt.style.use("cyberpunk")

dados_fechamento.plot(y="Bitcoin", use_index = True, legend = False)

plt.title("BTC/USD")

plt.savefig("Bitcoin.png", dpi = 300)
plt.show()

In [ ]:
# Criar os gráficos a partir dos dados

# Definindo o tema do grafico
plt.style.use("cyberpunk")

dados_fechamento.plot(y="Dólar", use_index = True, legend = False)

plt.title("USD/BRL")

plt.savefig("Dólar.png", dpi = 300)
plt.show()

In [ ]:
# Criar os gráficos a partir dos dados

# Definindo o tema do grafico
plt.style.use("cyberpunk")

dados_fechamento.plot(y="Ethereum", use_index = True, legend = False)

plt.title("ETH/USD")

plt.savefig("Ethereum.png", dpi = 300)
plt.show()

In [ ]:
# Criar os gráficos a partir dos dados

# Definindo o tema do grafico
plt.style.use("cyberpunk")

dados_fechamento.plot(y="SP500", use_index = True, legend = False)

plt.title("SP500/USD")

plt.savefig("SP500.png", dpi = 300)
plt.show()

In [ ]:
print(f'''
Prezado diretor, segue o relatório diário:

Bolsa:

No ano o SP500 está tendo uma rentabilidade de {interval_dicts["anual"]["sp500"]}%, 
enquanto no mês a rentabilidade é de {interval_dicts["mensal"]["sp500"]}%.

No último dia útil, o fechamento do SP500 foi de {interval_dicts["diario"]["sp500"]}%.

Dólar:

No ano o Dólar está tendo uma rentabilidade de {interval_dicts["anual"]["dolar"]}%, 
enquanto no mês a rentabilidade é de {interval_dicts["mensal"]["dolar"]}%.

No último dia útil, o fechamento do Dólar foi de {interval_dicts["diario"]["dolar"]}%.

Criptomoedas:

No ano, o Bitcoin está tendo uma rentabilidade de {interval_dicts["anual"]["bitcoin"]}%,
enquanto no mês a rentabilidade é de {interval_dicts["mensal"]["bitcoin"]}%.

Já o Ethereum, está tendo uma rentabilidade anual de {interval_dicts["anual"]["ethereum"]}%,
enquanto no mês a rentabilidade é de {interval_dicts["mensal"]["ethereum"]}%.

A respeito das rentabilidades diárias, Bitcoin terminou o dia retornando {interval_dicts["diario"]["bitcoin"]}%,
enquanto o Ethereum terminou o dia retornando {interval_dicts["diario"]["ethereum"]}%.

Abs,

O melhor estagiário do mundo

''')

In [23]:
# Maneira de enviar email pelo WINDOWS
outlook = win32.Dispatch('outlook.application')

mail = outlook.CreateItem(0)

mail.Subject = "Relatório Diário de " + datetime.now().strftime('%#d %b %Y %H:%M')
mail.To = "nformenton@gmail.com"
mail.Body = f'''Prezado diretor, segue o relatório diário:

Bolsa:

No ano o SP500 está tendo uma rentabilidade de {interval_dicts["anual"]["sp500"]}%, 
enquanto no mês a rentabilidade é de {interval_dicts["mensal"]["sp500"]}%.

No último dia útil, o fechamento do SP500 foi de {interval_dicts["diario"]["sp500"]}%.

Dólar:

No ano o Dólar está tendo uma rentabilidade de {interval_dicts["anual"]["dolar"]}%, 
enquanto no mês a rentabilidade é de {interval_dicts["mensal"]["dolar"]}%.

No último dia útil, o fechamento do Dólar foi de {interval_dicts["diario"]["dolar"]}%.

Criptomoedas:

No ano, o Bitcoin está tendo uma rentabilidade de {interval_dicts["anual"]["bitcoin"]}%,
enquanto no mês a rentabilidade é de {interval_dicts["mensal"]["bitcoin"]}%.

Já o Ethereum, está tendo uma rentabilidade anual de {interval_dicts["anual"]["ethereum"]}%,
enquanto no mês a rentabilidade é de {interval_dicts["mensal"]["ethereum"]}%.

A respeito das rentabilidades diárias, Bitcoin terminou o dia retornando {interval_dicts["diario"]["bitcoin"]}%,
enquanto o Ethereum terminou o dia retornando {interval_dicts["diario"]["ethereum"]}%.

Abs,

O melhor estagiário do mundo

'''

print(mail.Body)

# Windows
anexo_dolar = r"C:\Dev\Python - DB1\Dólar.png"
anexo_bitcoin = r"C:\Dev\Python - DB1\Bitcoin.png"
anexo_ethereum = r"C:\Dev\Python - DB1\Ethereum.png"
anexo_sp500 = r"C:\Dev\Python - DB1\SP500.png"
# 

# Linux
# anexo_dolar = r'/home/nicolas/dev/BOT.Camp - Python/Dólar.png'
# anexo_bitcoin = r'/home/nicolas/dev/BOT.Camp - Python/Bitcoin.png'
# anexo_ethereum = r'/home/nicolas/dev/BOT.Camp - Python/Ethereum.png'
# anexo_sp500 = r'/home/nicolas/dev/BOT.Camp - Python/SP500.png'
# 

mail.Attachments.Add(anexo_dolar)
mail.Attachments.Add(anexo_sp500)
mail.Attachments.Add(anexo_bitcoin)
mail.Attachments.Add(anexo_ethereum)

mail.Send()


com_error: (-2147221005, 'Invalid class string', None, None)

In [ ]:
outlook = win32.Dispatch('Outlook.Application')